In [1]:
'''
Notebook to analyze and compare across sweep runs 
'''

import os

import cmws
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from cmws import util
from cmws.examples.scene_understanding import data, render, run, plot
from cmws.examples.scene_understanding import util as scene3d_util 
import seaborn as sns
import numpy as np


# from: https://github.com/tuananhle7/continuous_mws/blob/master/cmws/examples/timeseries/plot.py#L244
def plot_with_error_bars(ax, x, data, **plot_kwargs):
    mid = np.nanmedian(data, axis=0)
    low = np.nanpercentile(data, 25, axis=0)
    high = np.nanpercentile(data, 75, axis=0)

    num_not_nan = np.count_nonzero(~np.isnan(mid))

    if num_not_nan > 0:
        lines = ax.plot(x[:num_not_nan], mid[:num_not_nan], **plot_kwargs)
        ax.fill_between(
            x[:num_not_nan],
            low[:num_not_nan],
            high[:num_not_nan],
            alpha=0.2,
            color=lines[0].get_color(),
        )

In [2]:
def get_analysis_plots(experiment_name, grid_sizes=[2, 3], cmws_version="cmws_2"):
    save_dir = f"../save/{experiment_name}"
    checkpoint_paths = []
    for config_name in sorted(os.listdir(save_dir)):
        checkpoint_paths.append(util.get_checkpoint_path(experiment_name, config_name, -1))

    for grid_size in grid_sizes:  # grid size
        fig, axs = plt.subplots(1, 2, figsize=(2 * 6, 1 * 4))

        colors = {cmws_version: "C0", "rws": "C1"}
        added_label = {k: False for k in colors.keys()}  # keep track of whether we've used alg as label
        for checkpoint_path in checkpoint_paths:
            checkpoint_path = f"../{checkpoint_path}"

            # Fix seed
            util.set_seed(1)

            if os.path.exists(checkpoint_path):
                # Load checkpoint
                try:
                    model, optimizer, stats, run_args = scene3d_util.load_checkpoint(
                        checkpoint_path, device="cpu"
                    )
                except:
                    continue

                if run_args.num_grid_cols != grid_size: continue

                generative_model, guide = model["generative_model"], model["guide"]
                num_iterations = len(stats.losses)

                if not added_label[run_args.algorithm]:
                    label = run_args.algorithm
                    added_label[run_args.algorithm] = True
                else:
                    label = None
                color = colors[run_args.algorithm]
                plot_kwargs = {"label": label, "color": color, "alpha": 0.8, "linewidth": 1.5}

                # Logp
                ax = axs[0]
                ax.plot([x[0] for x in stats.log_ps], [x[1] for x in stats.log_ps], **plot_kwargs)

                # KL
                ax = axs[1]
                ax.plot([x[0] for x in stats.kls], [x[1] for x in stats.kls], **plot_kwargs)
        ax = axs[0]
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Log p")

        ax = axs[1]
        ax.set_xlabel("Iteration")
        ax.set_ylabel("KL")
        ax.legend()
        for ax in axs:
            sns.despine(ax=ax, trim=True)
        util.save_fig(fig, f"{save_dir}/losses_{grid_size}.png", dpi=200)

    for grid_size in grid_sizes:  # grid size
        # Load
        x = []
        log_ps = {cmws_version: [], "cmws": [], "rws": []}
        kls = {cmws_version: [], "cmws": [], "rws": []}
        colors = {cmws_version: "C0", "rws": "C1"}
        for checkpoint_path in checkpoint_paths:
            checkpoint_path = f"../{checkpoint_path}"
            if os.path.exists(checkpoint_path):
                # Load checkpoint
                try:
                    model, optimizer, stats, run_args = scene3d_util.load_checkpoint(
                        checkpoint_path, device="cpu"
                    )
                except:
                    continue
                if run_args.num_grid_cols != grid_size: continue
                x_new = [x[0] for x in stats.log_ps]
                if len(x_new) > len(x):
                    x = x_new
                log_ps[run_args.algorithm].append([x[1] for x in stats.log_ps])
                kls[run_args.algorithm].append([x[1] for x in stats.kls])
        # Make numpy arrays
        max_len = len(x)
        if grid_size == 2:
            num_seeds = 5  # 10
        else:
            num_seeds = 5
        algorithms = [cmws_version, "rws"]
        log_ps_np = dict(
            [[algorithm, np.full((num_seeds, max_len), np.nan)] for algorithm in algorithms]
        )
        kls_np = dict([[algorithm, np.full((num_seeds, max_len), np.nan)] for algorithm in algorithms])
        for algorithm in algorithms:
            for seed in range(num_seeds):
                try:
                    log_p = log_ps[algorithm][seed]
                    kl = kls[algorithm][seed]
                except Exception:
                    log_p = []
                    kl = []
                log_ps_np[algorithm][seed][: len(log_p)] = log_p
                kls_np[algorithm][seed][: len(kl)] = kl

        # Plot
        fig, axs = plt.subplots(1, 2, figsize=(2 * 6, 1 * 4))
        for algorithm in algorithms:
            label = algorithm
            linestyle = "solid"
            color = colors[algorithm]
            plot_kwargs = {"color": color, "linestyle": linestyle, "label": label}

            # Logp
            log_p = log_ps_np[algorithm]
            ax = axs[0]
            plot_with_error_bars(ax, x, log_p, **plot_kwargs)

            # KL
            kl = kls_np[algorithm]
            ax = axs[1]
            plot_with_error_bars(ax, x, kl, **plot_kwargs)

        ax = axs[0]
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Log p")
        ax.legend()

        ax = axs[1]
        ax.set_xlabel("Iteration")
        ax.set_ylabel("KL")
        ax.legend()
        for ax in axs:
            sns.despine(ax=ax, trim=True)
        util.save_fig(fig, f"{save_dir}/losses_{grid_size}_overlay.png", dpi=200)

In [3]:
experiment_names = ["cmws_vs_rws_learnColor_shrink001","cmws_vs_rws_noColor_shrink001"]

for experiment_name in experiment_names: 
    get_analysis_plots(experiment_name, grid_sizes=[2, 3], cmws_version="cmws_2")

color status:  False
path:  /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/train.pt
23:50:33 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:302 | INFO: Loading dataset (test = False)...
23:50:33 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:306 | INFO: Dataset (test = False) loaded /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/train.pt
23:50:33 | /om/user/katiemc/continuous_mws/cmws/memory.py:20 | INFO: Initializing memory


100%|██████████| 100/100 [00:00<00:00, 1169.06it/s]

color status:  False
path:  /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/train.pt
23:50:34 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:302 | INFO: Loading dataset (test = False)...
23:50:34 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:306 | INFO: Dataset (test = False) loaded /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/train.pt
23:50:34 | /om/user/katiemc/continuous_mws/cmws/memory.py:20 | INFO: Initializing memory



100%|██████████| 100/100 [00:00<00:00, 1179.85it/s]


color status:  False
path:  /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/train.pt
23:50:34 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:302 | INFO: Loading dataset (test = False)...
23:50:34 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:306 | INFO: Dataset (test = False) loaded /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/train.pt
23:50:34 | /om/user/katiemc/continuous_mws/cmws/memory.py:20 | INFO: Initializing memory


...

color status:  True
path:  /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/colorless/train.pt
23:51:00 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:302 | INFO: Loading dataset (test = False)...
23:51:00 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:306 | INFO: Dataset (test = False) loaded /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/colorless/train.pt
23:51:00 | /om/user/katiemc/continuous_mws/cmws/memory.py:20 | INFO: Initializing memory



100%|██████████| 100/100 [00:00<00:00, 1163.89it/s]

color status:  True
path:  /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/colorless/train.pt
23:51:00 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:302 | INFO: Loading dataset (test = False)...
23:51:00 | /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data.py:306 | INFO: Dataset (test = False) loaded /om/user/katiemc/continuous_mws/cmws/examples/scene_understanding/data/1_1/colorless/train.pt
23:51:00 | /om/user/katiemc/continuous_mws/cmws/memory.py:20 | INFO: Initializing memory



100%|██████████| 100/100 [00:00<00:00, 1164.11it/s]
/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)
/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1114: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)
/home/katiemc/.conda/envs/cmws/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1390: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interpolation)


23:51:01 | /om/user/katiemc/continuous_mws/cmws/util.py:293 | INFO: Saved to ../save/cmws_vs_rws_noColor_shrink001/losses_3_overlay.png
